In [1]:
import os
from os.path import join
import tensorflow as tf
import pandas as pd
import numpy as np
from zipfile import ZipFile
from io import BytesIO
import PIL.Image

image_size = 224
num_channels = 3
num_classes = 120
num_input = image_size*image_size
learning_rate = 0.001

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

image_dir = 'C:/Users/User/Downloads/Capstone Project/train/'


dir_name = 'C:/Users/User/Downloads/Capstone Project/'
filenames1 = ZipFile('C:/Users/User/Downloads/Capstone Project/train.zip','r') 
 


labels = pd.read_csv("C:/Users/User/Downloads/Capstone Project/labels.csv")


In [ ]:
from tensorflow.python.keras.applications import VGG16

import ssl
ssl._create_default_https_context = ssl._create_unverified_context
model1 = VGG16(weights='imagenet',include_top=True)
    
#model1 = VGG16(weights='imagenet',include_top=False)

weights1 = model1.get_weights()

In [ ]:
X = tf.placeholder(tf.float32, [None, image_size,image_size,num_channels])
Y = tf.placeholder(tf.float32, [None, num_classes])
Z = tf.placeholder(tf.float32, [None, 4,4,512])
keep_prob = tf.placeholder(tf.float32)

In [ ]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [ ]:
def conv_net(x, weights):   
    
    x = tf.reshape(x, shape=[-1, image_size, image_size, 3])
    
    conv_1 = conv2d(x,weights[0],weights[1])
    conv_2 = conv2d(conv_1,weights[2],weights[3])
    max_1 = maxpool2d(conv_2)
    
    conv_3 = conv2d(max_1,weights[4],weights[5])
    conv_4 = conv2d(conv_3,weights[6],weights[7])
    max_2 = maxpool2d(conv_4)
    
    conv_5 = conv2d(max_2,weights[8],weights[9])
    conv_6 = conv2d(conv_5,weights[10],weights[11])
    conv_7 = conv2d(conv_6,weights[12],weights[13])
    max_3 = maxpool2d(conv_7)
    
    conv_8 = conv2d(max_3,weights[14],weights[15])
    conv_9 = conv2d(conv_8,weights[16],weights[17])
    conv_10 = conv2d(conv_9,weights[18],weights[19])
    max_4 = maxpool2d(conv_10)
    
    conv_11 = conv2d(max_4,weights[20],weights[21])
    conv_12 = conv2d(conv_11,weights[22],weights[23])
    conv_13 = conv2d(conv_12,weights[24],weights[25])
    max_5 = maxpool2d(conv_13)
    
    fc1 = tf.contrib.layers.flatten(max_5)
    fc1 = tf.add(tf.matmul(fc1, weights[26]), weights[27])
    fc1 = tf.nn.relu(fc1)
    
    fc2 = tf.add(tf.matmul(fc1, weights[28]), weights[29])
    fc2 = tf.nn.relu(fc2)
    
    fc3 = tf.add(tf.matmul(fc2, weights[30]), weights[31])
    fc3 = tf.nn.relu(fc3)
    
    return fc3

In [ ]:
pre_train = conv_net(X,weights1)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    imagevec = np.zeros([10222,1000])
    
    for j in range(10222):
        
        filename = BytesIO(filenames1.read(filenames1.namelist()[j+1]))
        image = PIL.Image.open(filename)
        image = image.resize((224, 224))
        image = np.array(image)
        image = np.clip(image/255.0, 0.0, 1.0)
        imagevec[j] = sess.run(pre_train, feed_dict = {X: image.reshape(1, 224, 224, 3)})

In [ ]:
label = pd.get_dummies(list(labels['breed']))
labels3 = np.dot(label.values,np.arange(1,121))

from sklearn import svm
from sklearn.model_selection import train_test_split

model = svm.SVC()

X_train, X_test, y_train, y_test = train_test_split(imagevec, labels3, test_size=0.33, random_state=42)
model.fit(X_train,y_train)

pred = model.predict(X_test)

correct = 0
length = len(X_test)

correct = pred == y_test
my_accuracy = (np.sum(correct) / length)*100
print ('svm LR Accuracy %: ', my_accuracy)